In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#use requests to get the data and then use beautiful soup so we can parse through it
oscars2019_raw = requests.get('https://www.oscars.org/oscars/ceremonies/2019').text

oscars2019_bs = BeautifulSoup(oscars2019_raw)

In [ ]:
#get all award groups
oscars2019_groups = oscars2019_bs.find_all('div',{'class':'view-grouping'})

#look at the first one
oscars2019_groups[0]

In [ ]:
#look at the last one
oscars2019_groups[-1]

In [ ]:
#count number of award groups
len(oscars2019_groups)

In [ ]:
#HTML tree to find more specific parents
oscars2019_parent_group = oscars2019_bs.find_all('div',{'id':'quicktabs-tabpage-honorees-0'})

#count groups matching the pattern; only 1
len(oscars2019_parent_group)

In [ ]:
#search within the parent group
oscars2019_true_groups = oscars2019_parent_group[0].find_all('div',{'class':'view-grouping'})
#number of award groups
len(oscars2019_true_groups)

In [ ]:
#loop to find award names in each group
for group in oscars2019_parent_group[0].find_all('div',{'class':'view-grouping-header'}):
    print(group.text)

In [ ]:
#find tag for "Actor in a Leading Role"
oscars2019_groups[0]

In [ ]:
#get parent
oscars2019_group0_parent = oscars2019_groups[0].parent

#find relevant children
oscars2019_group0_parent_children = oscars2019_group0_parent.find_all('div',{'class':'view-grouping'})

#count
len(oscars2019_group0_parent_children)

Look at relevant fields and output HTML output for content mapping

In [ ]:
oscars2019_true_groups[0]

In [ ]:
oscars2019_true_groups[0].find_all('div',{'class':"views-field views-field-field-actor-name"})

In [ ]:
oscars2019_true_groups[0].find_all('h4')

In [ ]:
oscars2019_true_groups[0].find_all('div',{'class':"views-field views-field-title"})

In [ ]:
oscars2019_true_groups[0].find_all('span')

In [ ]:
oscars2019_true_groups[-1].find_all('div',{'class':"views-field views-field-field-actor-name"})

In [ ]:
oscars2019_true_groups[-1].find_all('div',{'class':"views-field views-field-title"})

In [ ]:
oscars2019_true_groups[0]

In [ ]:
category = oscars2019_true_groups[0].find_all('h2')[0].text
print("The name of the category is:",category)

In [ ]:
names = []
for _nominee in oscars2019_true_groups[0].find_all('h4'):
    nominee_name = _nominee.text
    names.append(nominee_name)
    print("The name of a nominee is:",nominee_name)

In [ ]:
movies = []

for _movie in oscars2019_true_groups[0].find_all('span'):
    if len(_movie.text) > 0:
        movie_name = _movie.text.strip()
        movies.append(movie_name)
        print("The name of a movie is:",movie_name)

In [ ]:
#extract relevant info from subdivs
actor_nominees = oscars2019_true_groups[0].find_all('div',{'class':'views-row'})

for i,nominee in enumerate(actor_nominees):
    
    if i == 0:
        winner = 'Won'
    else:
        winner = 'Lost'
    
    #list of all subdivs
    subdivs = nominee.find_all('div')
    
    #first subdiv (for an actor) is the name
    name = subdivs[0].text.strip()
    
    #second subdiv (for an actor) is the movie name
    movie = subdivs[1].text.strip()
    
    print("{0} was nominated for \"{1}\" and {2}.".format(name,movie,winner))

In [ ]:
#reversing 'movie' and 'name' works for another award category (making sure code is sound)
original_screenplay_nominees = oscars2019_true_groups[-1].find_all('div',{'class':'views-row'})

for i,nominee in enumerate(original_screenplay_nominees):
    if i == 0:
        winner = 'Won'
    else:
        winner = 'Lost'
        
    subdivs = nominee.find_all('div')
    
    #movie and name reversed
    movie = subdivs[0].text.strip()
    name = subdivs[1].text.strip()
    
    print("{0} was nominated for \"{1}\" and {2}.".format(name,movie,winner))

In [ ]:
#new format of 'name' 'category' 'movie' and 'winner'
for group in oscars2019_true_groups:
    category = group.find_all('h2')[0].text
    
    for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
        if i == 0:
            winner = 'Won'
        else:
            winner = 'Lost'

        subdivs = nominee.find_all('div')
        name = subdivs[0].text.strip()
        movie = subdivs[1].text.strip()

        print("{0} was nominated in {1} for {2}\" and {3}.".format(name,category,movie,winner))

In [ ]:
for group in oscars2019_true_groups:
    category = group.find_all('h2')[0].text
    
    if 'Actor' in category or 'Actress' in category:
    
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            name = subdivs[0].text.strip()
            movie = subdivs[1].text.strip()

            print("{0} was nominated in {1} for \"{2}\" and {3}.".format(name,category,movie,winner))
            
    else:
        
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            movie = subdivs[0].text.strip()
            name = subdivs[1].text.strip()

            print("\"{0}\" was nominated in {1} for {2} and {3}.".format(name,category,movie,winner))

In [ ]:
#store info in 'nominees_2019' to turn into a dataframe
nominees_2019 = []

for group in oscars2019_true_groups:
    category = group.find_all('h2')[0].text
    
    if 'Actor' in category or 'Actress' in category:
    
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            name = subdivs[0].text.strip()
            movie = subdivs[1].text.strip()
            
            nominee_payload = {'Category':category,
                               'Name':name,
                               'Movie':movie,
                               'Year':2019, 
                               'Winner':winner}
            
            nominees_2019.append(nominee_payload)
            
    else:
        
        for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
            if i == 0:
                winner = 'Won'
            else:
                winner = 'Lost'

            subdivs = nominee.find_all('div')
            movie = subdivs[0].text.strip()
            name = subdivs[1].text.strip()

            nominee_payload = {'Category':category,
                               'Name':name,
                               'Movie':movie,
                               'Year':2018,
                               'Winner':winner}
            
            nominees_2019.append(nominee_payload)

In [ ]:
#pandas dataframe of 2019 nominees
nominees_df = pd.DataFrame(nominees_2019)
nominees_df.head(10)

In [ ]:
nominees_df.to_excel(r'C:\Users\Shailaja\Documents\college`\2019 spring\ban 172 - web analytics\final project\oscars_allyr.xlsx')

In [ ]:
#turn parser into a function to apply to other years' nominees
def parse_nominees(true_groups,year):
    nominees_list = []

    for group in true_groups:
        category = group.find_all('h2')[0].text

        if 'Actor' in category or 'Actress' in category:

            for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
                if i == 0:
                    winner = 'Won'
                else:
                    winner = 'Lost'

                subdivs = nominee.find_all('div')
                name = subdivs[0].text.strip()
                movie = subdivs[1].text.strip()
                
                nominee_payload = {'Category':category,
                                   'Name':name,
                                   'Movie':movie,
                                   'Year':year,
                                   'Winner':winner}

                nominees_list.append(nominee_payload)

        else:

            for i,nominee in enumerate(group.find_all('div',{'class':'views-row'})):
                if i == 0:
                    winner = 'Won'
                else:
                    winner = 'Lost'

                subdivs = nominee.find_all('div')
                movie = subdivs[0].text.strip()
                name = subdivs[1].text.strip()

                nominee_payload = {'Category':category,
                                   'Name':name,
                                   'Movie':movie,
                                   'Year':year,
                                   'Winner':winner}

                nominees_list.append(nominee_payload)
    
    return nominees_list

In [ ]:
#use this by including 'sleep' between each request; doesn't overwhlem the server (wasn't working without this)
from time import sleep

In [ ]:
print("The start of something.")
sleep(3)
print("The end of something.")

In [ ]:
#prints award pages from 2010-2020; 1 second request between URL retrieval
for year in range(2010,2020):
    sleep(1)
    print('https://www.oscars.org/oscars/ceremonies/{0}'.format(year))

In [ ]:
#create an empty list to store the data we get
all_years_nominees = dict()

for year in range(2010,2020):

    sleep(1) 
    
    #get raw HTML & use beautiful soup
    year_raw_html = requests.get('https://www.oscars.org/oscars/ceremonies/{0}'.format(year)).text
    year_soup_html = BeautifulSoup(year_raw_html)
    
    #get the parent group
    year_parent_group = year_soup_html.find_all('div',{'id':'quicktabs-tabpage-honorees-0'})
    
    #get the true groups under the parent group
    year_true_groups = year_parent_group[0].find_all('div',{'class':'view-grouping'})
    
    #use parsing function
    year_nominees = parse_nominees(year_true_groups,year)
    
    #convert the year_nominees to a df and add them to all_years_nominees
    all_years_nominees[year] = pd.DataFrame(year_nominees)

In [ ]:
#dataframe including all years
all_years_nominees_df = pd.concat(all_years_nominees)
all_years_nominees_df.reset_index(drop=True)

In [ ]:
all_years_nominees_df.to_excel(r'C:\Users\Shailaja\Documents\college\2019 spring\ban 172 - web analytics\final project\oscars_allyears.xlsx')